In [1]:
import numpy as np
import random
import time
import math
import keyboard
from IPython.display import clear_output

In [2]:
import multiprocess as mp

from numba import njit,jit,prange
from numba.core import types
from numba.typed import Dict

In [3]:
k=0.2/10*np.log(10)

In [8]:
@njit
def factorial(n):
    if n == 0 or n == 1:
        return 1
    else:
        result = 1
        for i in range(2, n+1):
            result *= i
        return result
@njit
def comb(n, k):
    if k > n:
        return 0
    return factorial(n) // (factorial(k) * factorial(n - k))

# Z_n is the mean number of timeslots until all segments are generated 
@njit
def Z(p,N):
    sol_ = 0
    for j in prange(1,N+1):
        term = comb(N,j) * (-1)**(j+1) / (1 - (1-p)**j)
        sol_ += term
    return sol_

@njit
def exp_subl(L,N):
    return Z(np.exp(-k*L/N),N)

In [16]:
def time_f(f,arg):
    start_time = time.time()
    f(*arg)
    end_time = time.time()
    elapsed_time = end_time - start_time
    return round(elapsed_time,5)

In [5]:
def test_speed_f(f):
    return f(i=500000,N=2,L=1,q=0.3,max_iter=1000000)

In [6]:
def prob_all_SaE(N,p,q,max_iter):
    shortest_sl = [(i,i+1) for i in range(N)]
    truth_vals = {(i,i+1):False for i in range(N)}
    truth_vals[(0,N)] = False
    counter = 0

    while True:
        counter += 1

        existing_links = np.sort([link for link in truth_vals if truth_vals[link]])
        for link in shortest_sl:
            big_link_exists = False
            if not truth_vals[link]:
                for big_link in existing_links:
                    if big_link[0]<=link[0] and link[-1]<=big_link[-1]:
                        big_link_exists = True
                        break
                if big_link_exists:
                    continue   
                
                if random.random() < p:
                    truth_vals[link] = True
        
        existing_links = np.sort([link for link in truth_vals if truth_vals[link]])

        temp_rep = 0
        all_linked = False
        while temp_rep != N:
            valid_links = [link for link in existing_links if link[0]==temp_rep]
            if len(valid_links) > 0:
                temp_rep = valid_links[0][-1]
            elif len(valid_links) == 0:
                all_linked = False
                break
        if temp_rep == N:
            all_linked = True

        if all_linked:
            for rep in range(1,N):
                sl_L = None
                sl_R = None
                for link in truth_vals:
                    if truth_vals[link]:
                        if rep == link[-1]:
                            sl_L = link
                        if rep == link[0]:
                            sl_R = link

                if sl_L != None and sl_R != None:
                    if random.random() < q: # succeed -> entanglement 
                        truth_vals[(sl_L[0],sl_R[-1])] = True
                        truth_vals[sl_L] = False
                        truth_vals[sl_R] = False
                    else: # fail
                        truth_vals[sl_L] = False
                        truth_vals[sl_R] = False
                else:
                    pass
        
        if truth_vals[(0,N)]:
            return counter
        
        # stop if iter > ...
        elif counter >= max_iter:
            return counter


def iter_SaE(i, N, L, q, max_iter):
    l = []
    p = np.exp(-k * L / N)
    for _temp_ in range(i):
        l.append(prob_all_SaE(N=N, p=p, q=q, max_iter=max_iter))
    return l

def iter_SaE(i:int, N:int, L:int, q:float, max_iter:int):
    l = np.empty(i, dtype=np.int64)
    p = np.exp(-k * L / N)
    for _temp_ in prange(i):
        l[_temp_] = prob_all_SaE(N=N, p=p, q=q, max_iter=max_iter)
    return l


In [7]:
heatmap_dict_big = {}

for L__ in range(10,200+10,10):

    for q__ in np.arange(0.1, 1.1, 0.1):
        q__ = round(q__,3)

        for N__ in range(1,10+1):
            # at longer lengths, direct link is never optimal
            if L__>200 and N__ == 1:
                continue
            elif q__<=0.6 and N__>20:
                continue
            elif q__<=0.4 and N__>15:
                continue
            elif q__<=0.2 and N__>10:
                continue
            elif q__==0.05 and L__<=650 and N__>6:
                continue
            elif q__==0.05 and N__>8:
                continue
            elif q__==0.1 and N__>3:
                continue
            
            # check if prev asc.
            # if previous 4 values increasing, terminate
            if (L__,q__,N__-4) in heatmap_dict_big.keys():
                prev4_l = [np.mean(heatmap_dict_big[(L__,q__,enn)]) for enn in (N__-4,N__-3,N__-2,N__-1)]
                if sorted(prev4_l) == prev4_l and len(set(prev4_l))==4:
                    break


            if (L__,q__,N__) not in heatmap_dict_big.keys():

                #print(f'generating on ({L__},{q__},{N__})')

                if q__ == 1:
                    heatmap_dict_big[(L__,q__,N__)] = [exp_subl(L__,N__)]
                elif q__ == 0.1 and L__ >= 600:
                    heatmap_dict_big[(L__,q__,N__)] = iter_SaE(i=1,N=N__,L=L__,q=q__,max_iter=5000000)
                elif q__ == 0.05:
                    heatmap_dict_big[(L__,q__,N__)] = iter_SaE(i=1,N=N__,L=L__,q=q__,max_iter=10000000)
                else:
                    heatmap_dict_big[(L__,q__,N__)] = iter_SaE(i=1,N=N__,L=L__,q=q__,max_iter=1000000)
                    
                
                #clear_output(wait=True)
                #print(f'({L__},{q__},{N__}) complete')

NameError: name 'exp_subl' is not defined

#### MP

In [2]:
def iter_SaE_worker(args):
    import np
    import random
    from heatmap.heatmap_func import prob_all_SaE

    k=0.2/10*np.log(10)

    N, L, q, max_iter = args
    p = np.exp(-k * L / N)
    return prob_all_SaE(N=N, p=p, q=q, max_iter=max_iter)

def iter_SaE_mp(i: int, N: int, L: int, q: float, max_iter: int):
    args = [(N, L, q, max_iter) for _ in range(i)]
    
    # Use Pool to parallelize the execution
    with mp.Pool(mp.cpu_count()) as pool:
        results = pool.map(iter_SaE_worker, args)

    return results

In [44]:
time_normal = time_f(iter_SaE,(30, 4, 600, 0.1, 100000))
time_mp = time_f(iter_SaE_mp,(30, 4, 600, 0.1, 100000))

print(f'no mp: {time_normal}')
print(f'w/ mp: {time_mp}')

no mp: 33.4268
w/ mp: 5.82027


In [12]:
heatmap_dict_big = {}

for L__ in range(10,200+10,10):

    for q__ in np.arange(0.1, 1.1, 0.1):
        q__ = round(q__,3)

        for N__ in range(1,10+1):
            # at longer lengths, direct link is never optimal
            if L__>200 and N__ == 1:
                continue
            elif q__<=0.6 and N__>20:
                continue
            elif q__<=0.4 and N__>15:
                continue
            elif q__<=0.2 and N__>10:
                continue
            elif q__==0.05 and L__<=650 and N__>6:
                continue
            elif q__==0.05 and N__>8:
                continue
            elif q__==0.1 and N__>3:
                continue
            
            # check if prev asc.
            # if previous 4 values increasing, terminate
            if (L__,q__,N__-4) in heatmap_dict_big.keys():
                prev4_l = [np.mean(heatmap_dict_big[(L__,q__,enn)]) for enn in (N__-4,N__-3,N__-2,N__-1)]
                if sorted(prev4_l) == prev4_l and len(set(prev4_l))==4:
                    break


            if (L__,q__,N__) not in heatmap_dict_big.keys():

                print(f'generating on ({L__},{q__},{N__})')

                if q__ == 1:
                    heatmap_dict_big[(L__,q__,N__)] = [exp_subl(L__,N__)]
                elif q__ == 0.1 and L__ >= 600:
                    heatmap_dict_big[(L__,q__,N__)] = iter_SaE_mp(i=1,N=N__,L=L__,q=q__,max_iter=5000000)
                elif q__ == 0.05:
                    heatmap_dict_big[(L__,q__,N__)] = iter_SaE_mp(i=1,N=N__,L=L__,q=q__,max_iter=10000000)
                else:
                    heatmap_dict_big[(L__,q__,N__)] = iter_SaE_mp(i=1,N=N__,L=L__,q=q__,max_iter=1000000)
                    
                
                clear_output(wait=True)
                print(f'({L__},{q__},{N__}) complete')

print(heatmap_dict_big)

(70,0.4,8) complete
generating on (70,0.4,9)


KeyboardInterrupt: 

#### njit

In [ ]:
key_type = types.UniTuple(types.int32, 2)

@njit
def prob_all_SaE_njit(N:int,p:float,q:float,max_iter:int):
    shortest_sl = [(i,i+1) for i in range(N)]

    truth_vals = Dict.empty(
        key_type=key_type, 
        value_type=types.bool,
    )
    for i in range(N):
        truth_vals[(i,i+1)] = False
    truth_vals[(0,N)] = False

    counter = 0
    
    while True:
        counter += 1

        existing_links = np.sort([link for link in truth_vals if truth_vals[link]])
        for link in shortest_sl:
            big_link_exists = False
            if not truth_vals[link]:
                for big_link in existing_links:
                    if big_link[0]<=link[0] and link[-1]<=big_link[-1]:
                        big_link_exists = True
                        break
                if big_link_exists:
                    continue   
                
                if np.random.random() < p:
                    truth_vals[link] = True
        
        existing_links = np.sort([link for link in truth_vals if truth_vals[link]])

        temp_rep = 0
        all_linked = False
        while temp_rep != N:
            valid_links = [link for link in existing_links if link[0]==temp_rep]
            if len(valid_links) > 0:
                temp_rep = valid_links[0][-1]
            elif len(valid_links) == 0:
                all_linked = False
                break
        if temp_rep == N:
            all_linked = True

        if all_linked:
            for rep in range(1,N):
                sl_L = (-1,-1)
                sl_R = (-1,-1)
                for link in truth_vals:
                    if truth_vals[link]:
                        if rep == link[-1]:
                            sl_L = link
                        if rep == link[0]:
                            sl_R = link

                if sl_L != (-1,-1) and sl_R != (-1,-1):
                    if np.random.random() < q: # succeed -> entanglement 
                        truth_vals[(sl_L[0],sl_R[-1])] = True
                        truth_vals[sl_L] = False
                        truth_vals[sl_R] = False
                    else: # fail
                        truth_vals[sl_L] = False
                        truth_vals[sl_R] = False
                else:
                    pass
        
        if truth_vals[(0,N)]:
            return counter
        
        # stop if iter > ...
        elif counter >= max_iter:
            return counter

@njit
def iter_SaE_njit(i:int, N:int, L:int, q:float, max_iter:int):
    l = np.empty(i, dtype=np.int64)
    p = np.exp(-k * L / N)
    for _temp_ in prange(i):
        l[_temp_] = prob_all_SaE_njit(N=N, p=p, q=q, max_iter=max_iter)
    return l



In [ ]:
key_type = types.Tuple((types.int64, types.float64, types.int64))

@njit
def generate_heatmap():
    
    heatmap_dict_big = Dict.empty(
        key_type=key_type,
        value_type=types.int64[:]
    )

    for i in prange(21):  # 21 iterations for L__ from 10 to 200
        L__ = (i + 1) * 10
        for j in prange(10):  # 10 iterations for q__ from 0.1 to 1.0
            q__ = round(0.1 * (j + 1), 3)
            for k in prange(10):  # 30 iterations for N__ from 1 to 30
                N__ = k + 1

                # Skipping conditions
                if (L__ > 200 and N__ == 1) or \
                   (q__ <= 0.6 and N__ > 20) or \
                   (q__ <= 0.4 and N__ > 15) or \
                   (q__ <= 0.2 and N__ > 10) or \
                   (q__ == 0.05 and L__ <= 650 and N__ > 6) or \
                   (q__ == 0.05 and N__ > 8) or \
                   (q__ == 0.1 and N__ > 3):
                    continue

                # Perform the computation
                if q__ == 1:
                    break
                    result = exp_subl(L__, N__)
                elif q__ == 0.1 and L__ >= 600:
                    result = iter_SaE_njit(1, N__, L__, q__, 5000000)
                elif q__ == 0.05:
                    result = iter_SaE_njit(1, N__, L__, q__, 10000000)
                else:
                    result = iter_SaE_njit(1, N__, L__, q__, 1000000)
                

                # Store the result in the dictionary
                heatmap_dict_big[(L__, q__, N__)] = result

    return heatmap_dict_big

# Running the parallelized heatmap generation
heatmap_dict_big = generate_heatmap()

NotImplementedError: 